<a href="https://colab.research.google.com/github/aAmohammadrezaaA/Retinal-Vessel-Segmentation_A-Computer-Vision-Technique/blob/main/implementation_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
from tabulate import tabulate
from prettytable import PrettyTable
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import AveragePooling2D,Conv2DTranspose,Input,Add,Conv2D, BatchNormalization,LeakyReLU, Activation, MaxPool2D, Dropout, Flatten, Dense,UpSampling2D,Concatenate,Softmax

In [75]:
patch_size=48
patch_num=1500
patch_threshold=25
BATCH_SIZE=64
LR=0.0003
channels=3

In [189]:
class LinearTransform(tf.keras.Model):
  def __init__(self, name="LinearTransform"):
    super(LinearTransform, self).__init__(self,name=name)

    self.conv_r=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.conv_g=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.conv_b=Conv2D(1,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.pool_rc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)
    self.pool_gc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)
    self.pool_bc=AveragePooling2D(pool_size=(patch_size,patch_size),strides=1)

    self.bn=BatchNormalization()
    self.sigmoid=Activation('sigmoid')
    self.softmax=Activation('softmax')

  def call(self, input,training=True):
    r,g,b=input[:,:,:,0:1],input[:,:,:,1:2],input[:,:,:,2:3]

    rs=self.conv_r(r)
    gs=self.conv_g(g)
    bs=self.conv_r(b)

    rc=tf.reshape(self.pool_rc(rs),[-1,1])
    gc=tf.reshape(self.pool_gc(gs),[-1,1])
    bc=tf.reshape(self.pool_bc(bs),[-1,1])

    merge=Concatenate(axis=-1)([rc,gc,bc])
    merge=tf.expand_dims(merge,axis=1)
    merge=tf.expand_dims(merge,axis=1)
    merge=self.softmax(merge)
    merge=tf.repeat(merge,repeats=48,axis=2)
    merge=tf.repeat(merge,repeats=48,axis=1)

    r=r*(1+self.sigmoid(rs))
    g=g*(1+self.sigmoid(gs))
    b=b*(1+self.sigmoid(bs))

    output=self.bn(merge[:,:,:,0:1]*r+merge[:,:,:,1:2]*g+merge[:,:,:,2:3]*b,training=training)
    return output

class ResBlock(tf.keras.Model):
  def __init__(self,out_ch,residual_path=False,stride=1):
    super(ResBlock,self).__init__(self)
    self.residual_path=residual_path

    self.conv1=Conv2D(out_ch,kernel_size=3,strides=stride,padding='same', use_bias=False,data_format="channels_last")
    self.bn1=BatchNormalization()
    self.relu1=LeakyReLU()#Activation('leaky_relu')

    self.conv2=Conv2D(out_ch,kernel_size=3,strides=1,padding='same', use_bias=False,data_format="channels_last")
    self.bn2=BatchNormalization()

    if residual_path:
      self.conv_shortcut=Conv2D(out_ch,kernel_size=1,strides=stride,padding='same',use_bias=False)
      self.bn_shortcut=BatchNormalization()

    self.relu2=LeakyReLU()#Activation('leaky_relu')

  def call(self,x,training=True):
    xs=self.relu1(self.bn1(self.conv1(x),training=training))
    xs=self.bn2(self.conv2(xs),training=training)

    if self.residual_path:
      x=self.bn_shortcut(self.conv_shortcut(x),training=training)
    #print(x.shape,xs.shape)
    xs=x+xs
    return self.relu2(xs)

class Unet(tf.keras.Model):
  def __init__(self):
    super(Unet,self).__init__(self)
    self.conv_init=LinearTransform()
    self.resinit=ResBlock(16,residual_path=True)
    self.up_sample=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resup=ResBlock(32,residual_path=True)

    self.pool1=MaxPool2D(pool_size=(2,2))

    self.resblock_down1=ResBlock(64,residual_path=True)
    self.resblock_down11=ResBlock(64,residual_path=False)
    self.pool2=MaxPool2D(pool_size=(2,2))

    self.resblock_down2=ResBlock(128,residual_path=True)
    self.resblock_down21=ResBlock(128,residual_path=False)
    self.pool3=MaxPool2D(pool_size=(2,2))

    self.resblock_down3=ResBlock(256,residual_path=True)
    self.resblock_down31=ResBlock(256,residual_path=False)
    self.pool4=MaxPool2D(pool_size=(2,2))

    self.resblock=ResBlock(512,residual_path=True)

    self.unpool3=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock_up3=ResBlock(256,residual_path=True)
    self.resblock_up31=ResBlock(256,residual_path=False)

    self.unpool2=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock_up2=ResBlock(128,residual_path=True)
    self.resblock_up21=ResBlock(128,residual_path=False)

    self.unpool1=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock_up1=ResBlock(64,residual_path=True)

    self.unpool_final=UpSampling2D(size=(2,2),interpolation='bilinear')
    self.resblock2=ResBlock(32,residual_path=True)

    self.pool_final=MaxPool2D(pool_size=(2,2))
    self.resfinal=ResBlock(32)

    self.conv_final=Conv2D(1,kernel_size=1,strides=1,padding='same',use_bias=False)
    self.bn_final=BatchNormalization()
    self.act=Activation('sigmoid')

  def call(self,x,training=True):
    x_linear=self.conv_init(x,training=training)
    x=self.resinit(x_linear,training=training)
    x=self.up_sample(x)
    x=self.resup(x,training=training)

    stage1=self.pool1(x)
    stage1=self.resblock_down1(stage1,training=training)
    stage1=self.resblock_down11(stage1,training=training)

    stage2=self.pool2(stage1)
    stage2=self.resblock_down2(stage2,training=training)
    stage2=self.resblock_down21(stage2,training=training)

    stage3=self.pool3(stage2)
    stage3=self.resblock_down3(stage3,training=training)
    stage3=self.resblock_down31(stage3,training=training)

    stage4=self.pool4(stage3)
    stage4=self.resblock(stage4,training=training)

    stage3=Concatenate(axis=3)([stage3,self.unpool3(stage4)])
    stage3=self.resblock_up3(stage3,training=training)
    stage3=self.resblock_up31(stage3,training=training)

    stage2=Concatenate(axis=3)([stage2,self.unpool2(stage3)])
    stage2=self.resblock_up2(stage2,training=training)
    stage2=self.resblock_up21(stage2,training=training)

    stage1=Concatenate(axis=3)([stage1,self.unpool1(stage2)])
    stage1=self.resblock_up1(stage1,training=training)

    x=Concatenate(axis=3)([x,self.unpool_final(stage1)])
    x=self.resblock2(x,training=training)

    x=self.pool_final(x)
    x=self.resfinal(x,training=training)

    seg_result=self.act(self.bn_final(self.conv_final(x),training=training))

    return x_linear,seg_result
class Unet2(tf.keras.Model):
  def __init__(self):
    super(Unet2,self).__init__(self)
    self.conv_init=LinearTransform()
    #self.resinit=ResBlock(16,residual_path=True)
    self.resinit=Conv2D(16,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.up_sample=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resup=ResBlock(32,residual_path=True)
    self.resup=Conv2D(32,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.pool1=MaxPool2D(pool_size=(2,2))

    #self.resblock_down1=ResBlock(64,residual_path=True)
    self.resblock_down1=Conv2D(64,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_down11=ResBlock(64,residual_path=False)
    self.resblock_down11=Conv2D(64,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.pool2=MaxPool2D(pool_size=(2,2))

    #self.resblock_down2=ResBlock(128,residual_path=True)
    self.resblock_down2=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_down21=ResBlock(128,residual_path=False)
    self.resblock_down21=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.pool3=MaxPool2D(pool_size=(2,2))

    #self.resblock_down3=ResBlock(256,residual_path=True)
    self.resblock_down3=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_down31=ResBlock(256,residual_path=False)
    self.resblock_down31=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)
    self.pool4=MaxPool2D(pool_size=(2,2))

    #self.resblock=ResBlock(512,residual_path=True)
    self.resblock=Conv2D(512,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool3=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock_up3=ResBlock(256,residual_path=True)
    self.resblock_up3=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_up31=ResBlock(256,residual_path=False)
    self.resblock_up31=Conv2D(256,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool2=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock_up2=ResBlock(128,residual_path=True)
    self.resblock_up2=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)
    #self.resblock_up21=ResBlock(128,residual_path=False)
    self.resblock_up21=Conv2D(128,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool1=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock_up1=ResBlock(64,residual_path=True)
    self.resblock_up1=Conv2D(64,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.unpool_final=UpSampling2D(size=(2,2),interpolation='bilinear')
    #self.resblock2=ResBlock(32,residual_path=True)
    self.resblock2=Conv2D(32,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.pool_final=MaxPool2D(pool_size=(2,2))
    #self.resfinal=ResBlock(32)
    self.resfinal=Conv2D(32,kernel_size=3,strides=1,padding='same',use_bias=False)

    self.conv_final=Conv2D(1,kernel_size=1,strides=1,padding='same',use_bias=False)
    self.bn_final=BatchNormalization()
    self.act=Activation('sigmoid')

  def call(self,x,training=True):
    x_linear=self.conv_init(x,training=training)
    x=self.resinit(x_linear,training=training)
    x=self.up_sample(x)
    x=self.resup(x,training=training)

    stage1=self.pool1(x)
    stage1=self.resblock_down1(stage1,training=training)
    stage1=self.resblock_down11(stage1,training=training)

    stage2=self.pool2(stage1)
    stage2=self.resblock_down2(stage2,training=training)
    stage2=self.resblock_down21(stage2,training=training)

    stage3=self.pool3(stage2)
    stage3=self.resblock_down3(stage3,training=training)
    stage3=self.resblock_down31(stage3,training=training)

    stage4=self.pool4(stage3)
    stage4=self.resblock(stage4,training=training)

    stage3=Concatenate(axis=3)([stage3,self.unpool3(stage4)])
    stage3=self.resblock_up3(stage3,training=training)
    stage3=self.resblock_up31(stage3,training=training)

    stage2=Concatenate(axis=3)([stage2,self.unpool2(stage3)])
    stage2=self.resblock_up2(stage2,training=training)
    stage2=self.resblock_up21(stage2,training=training)

    stage1=Concatenate(axis=3)([stage1,self.unpool1(stage2)])
    stage1=self.resblock_up1(stage1,training=training)

    x=Concatenate(axis=3)([x,self.unpool_final(stage1)])
    x=self.resblock2(x,training=training)

    x=self.pool_final(x)
    x=self.resfinal(x,training=training)

    seg_result=self.act(self.bn_final(self.conv_final(x),training=training))

    return x_linear,seg_result

In [172]:
cols = ["#", "patch_size", "patch_num", "patch_threshold", "batch_size", "learning_rate"]
rows = [[ "value", patch_size, patch_num, patch_threshold, BATCH_SIZE, LR],
        ["comment", "(48*48) windows", "number of windows", "threshold for the patch, the smaller threshoold, the less vessel in the patch", "batch", "LR"]]

print(tabulate(rows, headers=cols,tablefmt="fancy_grid"))

╒═════════╤═════════════════╤═══════════════════╤═══════════════════════════════════════════════════════════════════════════════╤══════════════╤═════════════════╕
│ #       │ patch_size      │ patch_num         │ patch_threshold                                                               │ batch_size   │ learning_rate   │
╞═════════╪═════════════════╪═══════════════════╪═══════════════════════════════════════════════════════════════════════════════╪══════════════╪═════════════════╡
│ value   │ 48              │ 1500              │ 25                                                                            │ 64           │ 0.0003          │
├─────────┼─────────────────┼───────────────────┼───────────────────────────────────────────────────────────────────────────────┼──────────────┼─────────────────┤
│ comment │ (48*48) windows │ number of windows │ threshold for the patch, the smaller threshoold, the less vessel in the patch │ batch        │ LR              │
╘═════════╧═══════════

In [190]:
input_shape = (patch_size, patch_size, channels)  # Adjusting the shape
batch_size = BATCH_SIZE

# random input tensor for testing
input_tensor = tf.random.normal((batch_size,) + input_shape)

model_unet1=Unet()
model_unet2=Unet2()
linear_output1, seg_result1 = model_unet1(input_tensor)
linear_output2, seg_result2 = model_unet2(input_tensor)
#model.summary(line_length=110)
#model_unet1.count_params()
#model_unet2.count_params()

In [193]:
train_loss_bc=0.0417;train_acc_bc=0.9228;train_f1_bc=0.5987;train_sp_bc=0.8952;train_se_bc=0.8952;train_precision_bc=0.894;train_auroc_bc=0.8952
val_loss_bc=0.2181;val_acc_bc=0.8011;val_f1_bc=0.4935;val_sp_bc=0.8060;val_se_bc=0.8060;val_precision_bc=0.8054;val_auroc_bc=0.8060
trained_till_epoch_bc=40; lowest_val_loss_on_epoch_bc=14

train_loss_d=0.0782; train_acc_d=0.9218; train_f1_d=0.5979; train_sp_d=0.8926; train_se_d=0.8926; train_precision_d=0.8920; train_auroc_d=0.8926
val_loss_d=0.1992; val_acc_d=0.8008; val_f1_d=0.4933; val_sp_d=0.8016; val_se_d=0.8016; val_precision_d=0.8011; val_auroc_d=0.8016
trained_till_epoch_d="Nan"; lowest_val_loss_on_epoch_d="Nan"

train_loss_f=170.6789; train_acc_f=0.8527; train_f1_f=0.5390; train_sp_f=0.7866; train_se_f=0.7040; train_precision_f=0.7859; train_auroc_f=0.7866
val_loss_f=4232.2256; val_acc_f=0.7367; val_f1_f=0.4396; val_sp_f=0.7040; val_se_f=0.8016; val_precision_f=0.7028; val_auroc_f=0.7040
trained_till_epoch_f="Nan"; lowest_val_loss_on_epoch_f="Nan"

train_loss_u2_bc=0.0693; train_acc_u2_bc=0.8881; train_f1_u2_bc=0.5682; train_sp_u2_bc=0.8840; train_se_u2_bc=0.8840; train_precision_u2_bc=0.8833; train_auroc_u2_bc=0.8840
val_loss_u2_bc=0.2361; val_acc_u2_bc=0.7802; val_f1_u2_bc=0.4762; val_sp_u2_bc=0.8205; val_se_u2_bc=0.8205; val_precision_u2_bc=0.8201; val_auroc_u2_bc=0.8205
trained_till_epoch_u2_bc=71; lowest_val_loss_on_epoch_u2_bc=33;

x = PrettyTable()
x.field_names = [f"Config of Net", "Net", "Number of parameters", "comments"]
x.add_row(["report", "UNet1", model_unet1.count_params(), "The Unet architecture which uses residual blocks and residual path and skip connections"])
x.add_row(["report", "UNet2", model_unet2.count_params(), "The same UNet, but replacing all the res blocks with Conv2D"])
print(x)
x = PrettyTable()
x.field_names = [f"Metrics on train data", "Net", "loss_function", "trained_till_epoch", f"lowest val_loss which was on epoch", "train_loss", "train_acc", "train_f1", "train_specificity", "train_sensitivity", "train_precision", "train_auroc"]
x.add_row(["report", "UNet1", "binary_crossentropy", trained_till_epoch_bc, lowest_val_loss_on_epoch_bc, train_loss_bc, train_acc_bc, train_f1_bc, train_sp_bc, train_se_bc, train_precision_bc, train_auroc_bc])
x.add_row(["report", "UNet1", "dice_loss", trained_till_epoch_d, lowest_val_loss_on_epoch_d, train_loss_d, train_acc_d, train_f1_d, train_sp_d, train_se_d, train_precision_d, train_auroc_d])
x.add_row(["report", "UNet1", "focal_loss", trained_till_epoch_f, lowest_val_loss_on_epoch_f, train_loss_f, train_acc_f, train_f1_f, train_sp_f, train_se_f, train_precision_f, train_auroc_f])
x.add_row(["report", "UNet2", "binary_crossentropy", trained_till_epoch_u2_bc, lowest_val_loss_on_epoch_u2_bc, train_loss_u2_bc, train_acc_u2_bc, train_f1_u2_bc, train_sp_u2_bc, train_se_u2_bc, train_precision_u2_bc, train_auroc_u2_bc])
print(x)


x = PrettyTable()
x.field_names = [f"Metrics on validation data", "Net", "loss_function", "trained_till_epoch", "lowest val_loss which was on epoch", "val_loss", "val_acc", "val_f1", "val_specificity", "val_sensitivity", "val_precision", "val_auroc"]
x.add_row(["report", "UNet1", "binary_crossentropy", trained_till_epoch_bc, lowest_val_loss_on_epoch_bc, val_loss_bc, val_acc_bc, val_f1_bc, val_sp_bc, val_se_bc, val_precision_bc, val_auroc_bc])
x.add_row(["report", "UNet1", "dice_loss", trained_till_epoch_d, lowest_val_loss_on_epoch_d, val_loss_d, val_acc_d, val_f1_d, val_sp_d, val_se_d, val_precision_d, val_auroc_d])
x.add_row(["report", "UNet1", "focal_loss", trained_till_epoch_f, lowest_val_loss_on_epoch_f, val_loss_f, val_acc_f, val_f1_f, val_sp_f, val_se_f, val_precision_f, val_auroc_f])
x.add_row(["report", "UNet2", "binary_crossentropy", trained_till_epoch_u2_bc, lowest_val_loss_on_epoch_u2_bc, val_loss_u2_bc, val_acc_u2_bc, val_f1_u2_bc, val_sp_u2_bc, val_se_u2_bc, val_precision_u2_bc, val_auroc_u2_bc])
print(x)


+---------------+-------+----------------------+-----------------------------------------------------------------------------------------+
| Config of Net |  Net  | Number of parameters |                                         comments                                        |
+---------------+-------+----------------------+-----------------------------------------------------------------------------------------+
|     report    | UNet1 |       11351706       | The Unet architecture which uses residual blocks and residual path and skip connections |
|     report    | UNet2 |       5442250        |               The same UNet, but replacing all the res blocks with Conv2D               |
+---------------+-------+----------------------+-----------------------------------------------------------------------------------------+
+-----------------------+-------+---------------------+--------------------+------------------------------------+------------+-----------+----------+-----------------